Data Scientist and Data Analyst jobs require a specialized set of skills. I want to identify the most in demand skills in the Glassdoor job descriptions for a Data Scientist and Data Analyst, so I know which ones  to focus on developing first to be a more highly rated candidate during my job search. Understanding where these jobs are located and in which industries will allow me to more efficiently focus my search in the appropriate state and industry to increase my odds of getting a job as a Data Scientist or Data Analyst.

In [ ]:
# Import Pandas and Numpy.
# Load data from .CSV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from collections import Counter
import lightgbm as lgb
import re
df = pd.read_csv('/Users/johnharmer/Library/CloudStorage/GoogleDrive-jwharmeriv@gmail.com/My Drive/Springboard - Data Science/Capstone Projects/Capstone 2/glassdoor_jobs.csv')

In [ ]:
df.head()

<h1 style="color: blue;"><b>DATA CLEANING</b></h1>

In [ ]:
# Filter job titles for 'Data Scientist' or 'Data Analyst'
df_filtered = df[df['Job Title'].str.contains('Data Scientist|Data Analyst', case=False, na=False)].copy()

In [ ]:
# Filter job titles for 'Data Scientist' or 'Data Analyst'
df_filtered.loc[:, 'State'] = df_filtered['Location'].apply(
    lambda x: 'Remote' if 'remote' in x.lower() else x.split(',')[-1].strip())

In [ ]:
# Extract the state from the Location column, treating "Remote" as its own category
df_filtered['State'] = df_filtered['Location'].apply(lambda x: 'Remote' if 'remote' in x.lower() else x.split(',')[-1].strip())

# Mapping of full state names to their 2-letter abbreviations
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
    'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Function to map full state names to abbreviations
def map_state_to_abbreviation(state):
    state = state.strip()
    if state in state_abbrev:
        return state_abbrev[state]
    return state

# Drop unwanted entries
df_filtered = df_filtered[~df_filtered['State'].isin(['United States', '-1', 'Point Loma', 'New York State', 'Manhattan'])]

# Apply the state abbreviation mapping (keeping 'Remote' as is)
df_filtered['State'] = df_filtered['State'].apply(lambda x: x if x == 'Remote' else map_state_to_abbreviation(x))

In [ ]:
df_filtered.head()

In [ ]:
# Common skills, technologies, and software to look for in job descriptions
skills_list = ['Python', 'SQL', 'Excel', 'R', 'Tableau', 'Power BI', 'Machine Learning', 
               'Deep Learning', 'Statistics', 'Data Mining', 'Hadoop', 'Spark', 
               'TensorFlow', 'Keras', 'scikit-learn', 'NLP', 'AWS', 'Azure', 'Cloud', 'Data Analysis']

# Define a function to extract skills from the job descriptions
def extract_skills(description):
    found_skills = set()
    for skill in skills_list:
        if re.search(r'\b' + re.escape(skill) + r'\b', description, re.IGNORECASE):
            found_skills.add(skill)
    return ', '.join(found_skills)

# Apply the function to the Job Description column and create a new 'Extracted Skills' column
df_filtered['Extracted Skills'] = df_filtered['Job Description'].apply(extract_skills)

# Display the updated dataframe with the new 'Extracted Skills' column
print(df_filtered[['Job Title', 'State', 'Extracted Skills']].head())

In [ ]:
# Concatenate all the extracted skills into one list
all_skills = df_filtered['Extracted Skills'].str.split(', ').explode()

# Count the occurrences of each skill
skill_counts = Counter(all_skills)

# Convert the Counter object to a DataFrame for easier plotting
skills_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Count']).sort_values(by='Count', ascending=False)

# Display the skill counts to the user
print(skills_df)

In [ ]:
# Remove empty skill entries and plot the skill counts
skills_df = skills_df[skills_df['Skill'] != '']

# Create bar plot to show Skills count
plt.figure(figsize=(10, 6))
plt.barh(skills_df['Skill'], skills_df['Count'], color='skyblue')
plt.xlabel('Count')
plt.ylabel('Skill')
plt.title('Skill Frequency in Job Descriptions')
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.show()

<b><h3>Most In-Demand Skills</b></h3>
<b>Top 15 Skills:</b> Machine Learning, Python, Statistics, R, Data Analysis, Data Mining, SQL, Cloud, Tableau, NLP, Excel, Deep Learning, Spark, TensorFlow, AWS<br>

In [ ]:
# Count how many unique job titles are in the df.
job_titles = df['Job Title'].value_counts()
print(job_titles)

In [ ]:
# Count occurrences of 'Data Scientist' and 'Data Analyst' separately
data_scientist_count = df_filtered['Job Title'].str.contains('Data Scientist', case=False, na=False).sum()
data_analyst_count = df_filtered['Job Title'].str.contains('Data Analyst', case=False, na=False).sum()

# Display counts
print(data_scientist_count, data_analyst_count)

# Create a DataFrame to hold the counts for plotting
job_title_data = pd.DataFrame({
    'Job Title': ['Data Scientist', 'Data Analyst'],
    'Count': [data_scientist_count, data_analyst_count]})

# Create a bar plot to visualize the counts
plt.figure(figsize=(6, 4))
plt.bar(job_title_data['Job Title'], job_title_data['Count'], color=['skyblue', 'lightgreen'])
plt.ylabel('Number of Positions')
plt.title('Count of Job Titles Containing "Data Scientist" or "Data Analyst"')
plt.show()

In [ ]:
# Count the number of unique industries in the df_filtered.
unique_industries_count = df_filtered['Industry'].nunique()

unique_industries_count

In [ ]:
# Count the occurrences of each Industry.
unique_industries_count = df_filtered['Industry'].value_counts()
unique_industries_count

In [ ]:
# Remove industries where the value is '-1'
industry_counts_cleaned = unique_industries_count[unique_industries_count.index != '-1']

# Print the cleaned industry counts with the industry names as the index
print(industry_counts_cleaned)

In [ ]:
# Plot the counts of each industry to help identify which industry is more likely to be hiring.
plt.figure(figsize=(10, 10))
industry_counts_cleaned.plot(kind='barh', color='lightblue')
plt.xlabel('Number of Positions')
plt.ylabel('Industry')
plt.title('Industry Hiring Counts')
plt.gca().invert_yaxis()  # Invert y-axis for readability
plt.grid(True, which='both', axis='both', linestyle='--', linewidth=0.5)
plt.show()

<b><h3>RESULTS</b></h3>
1226 jobs with job titles containing Data Scientist(1192) or Data Analyst(32).<br>

<b><h4>Top Industries Hiring:</b></h4>
Information Technology, Finance, Healthcare, Enterprise Software & Network Solutions, Aerospace & Defense.
IT support services has the most job openings.

In [ ]:
# Count how many jobs are located in each state, including remote positions
jobs_by_state = df_filtered['State'].value_counts()

# Display the results using standard print
print(jobs_by_state)

In [ ]:
# Plot a bar chart to show job counts by state
plt.figure(figsize=(10, 8))
jobs_by_state.plot(kind='barh', color='skyblue')
plt.xlabel('Number of Jobs')
plt.ylabel('State')
plt.title('Job Counts by State')
plt.gca().invert_yaxis()  # Invert y-axis for readability
plt.grid(True, which='both', axis='both', linestyle='--', linewidth=0.5)
plt.show()

<h3><B>Top 10 states with job openings</B></h3>
Remote    218<br>
CA        195<br>
VA        105<br>
TX         77<br>
NJ         77<br>
MA         59<br>
CO         39<br>
PA         38<br>
NC         36<br>
MI         30<br>

In [ ]:
# Function to clean and parse the salary column data
def clean_salary(salary):
    if 'K' in salary:
        salary = salary.replace('K', '').replace('$', '').replace('Employer Provided Salary:', '').replace('(Glassdoor est.)', '').strip()
        if '-' in salary:
            # Split into minimum and maximum salary and compute the average
            min_salary, max_salary = salary.split('-')
            avg_salary = (int(min_salary) + int(max_salary)) / 2
            return avg_salary
        else:
            # Single salary value
            return float(salary.strip())

In [ ]:
# Apply the clean_salary function to the Salary Estimate column
df_filtered['Avg Salary'] = df_filtered['Salary Estimate'].apply(clean_salary)

# Fill NaNs with a placeholder value (e.g., 0) and convert to integers
df_filtered['Avg Salary'].fillna(0, inplace=True)
df_filtered['Avg Salary'] = df_filtered['Avg Salary'].astype(int)

# Display 'Salary Estimate' and 'Avg Salary' to confirm.
df_filtered[['Salary Estimate', 'Avg Salary']]

In [ ]:
# Helper function to convert revenue to integer
def convert_to_integer(value):
    if 'M' in value:
        return int(float(value.replace('M', '').replace('+', '').replace('Less than ', '0.')) * 1e6)
    elif 'B' in value:
        return int(float(value.replace('B', '').replace('+', '').replace('Less than ', '0.')) * 1e9)
    return int(value.replace('+', '').replace('Less than ', '0.'))

# Function to parse and clean the revenue data with handling for mixed units and special cases
def parse_revenue(revenue):
    if 'Unknown' in revenue or 'Non-Applicable' in revenue:
        return None
    revenue = revenue.replace('(USD)', '').replace('$', '').replace(' million', 'M').replace(' billion', 'B').strip()
    if ' to ' in revenue:
        revenue_range = revenue.split(' to ')
        if 'M' in revenue_range[0] and 'B' in revenue_range[1]:
            min_revenue = float(revenue_range[0].replace('M', '').replace('+', '').replace('Less than ', '0.')) * 1e6
            max_revenue = float(revenue_range[1].replace('B', '').replace('+', '').replace('Less than ', '0.')) * 1e9
        elif 'B' in revenue_range[0] and 'M' in revenue_range[1]:
            min_revenue = float(revenue_range[0].replace('B', '').replace('+', '').replace('Less than ', '0.')) * 1e9
            max_revenue = float(revenue_range[1].replace('M', '').replace('+', '').replace('Less than ', '0.')) * 1e6
        else:
            min_revenue = convert_to_integer(revenue_range[0])
            max_revenue = convert_to_integer(revenue_range[1])
        return (min_revenue, max_revenue)
    elif 'M' in revenue or 'B' in revenue:
        revenue = convert_to_integer(revenue)
        return (revenue, revenue)
    return None

In [ ]:
# Apply the parsing function to the Revenue column
df_filtered['parsed_revenue'] = df_filtered['Revenue'].apply(parse_revenue)

df_filtered['parsed_revenue'] = df_filtered['Revenue'].apply(parse_revenue)

# Extract min and max revenue
df_filtered['Min Revenue'] = df_filtered['parsed_revenue'].apply(lambda x: x[0] if x is not None else None)
df_filtered['Max Revenue'] = df_filtered['parsed_revenue'].apply(lambda x: x[1] if x is not None else None)

# Drop the parsed_revenue column
df_filtered.drop(columns=['parsed_revenue'], inplace=True)

# Display the cleaned revenue ranges
df_filtered[['Revenue', 'Min Revenue', 'Max Revenue']].head()

In [ ]:
# Display summary statistics for numerical features
summary_stats_numerical = df_filtered.describe()

# Display summary statistics for categorical features
summary_stats_categorical = df_filtered.describe(include=['O'])

# Check for missing values
missing_values = df_filtered.isnull().sum()

summary_stats_numerical, summary_stats_categorical, missing_values

<b><h3>Inferential Statistics</b></h3>
<B>Hypotheses:</B><br>
<b>H0 (Null Hypothesis):</b> There is no significant difference in the average salary estimate between industries.<br>
<b>H1 (Alternative Hypothesis):</b> There is a significant difference in the average salary estimate between industries.

In [ ]:
# Scatter plot of average salary estimate vs. industry
plt.figure(figsize=(12, 10))
sns.scatterplot(data=df_filtered, x='Industry', y='Avg Salary')
plt.title('Relationship Between Avg Salary Estimate and Industry')
plt.xlabel('Industry')
plt.ylabel('Avg Salary Estimate')
plt.xticks(rotation=90)
plt.show()

Companies from the internet and software industires tend to pay higher wages. That being said, most of the industires are paying ~$100K or more for Data Scientist and Data Analyst jobs.

In [ ]:
# Filter data for box plot visualization
job_ratings = df_filtered[df_filtered['Job Title'].str.contains('Data Scientist|Data Analyst', case=False, na=False)]

# Box plot of Rating by job title
plt.figure(figsize=(16, 8))
sns.boxplot(x='Job Title', y='Rating', data=df_filtered)
plt.title('Ratings by Job Title')
plt.xlabel('Job Title')
plt.ylabel('Company Rating')
plt.xticks(rotation=45)
plt.show()

The box plot shows the distribution of company ratings across all Data Scientist and Data Analyst jobs. Both roles have a similar distribution of ratings with the mean rating around 3.8. This indicates that companies hiring for these rols generally have good ratings.

In [ ]:
# Box plot of ratings by state
plt.figure(figsize=(12,8))
sns.boxplot(x='State', y='Rating', data=df_filtered)
plt.title('Company Ratings by State')
plt.xlabel('State')
plt.ylabel('Company Rating')
plt.xticks(rotation=45)
plt.show()

The box plot shows the distribution of company ratings across all states with Data Scientist and Data Analyst roles available. Most states have median ratings of 3.5 to 4.1, indicating generally positive company ratings across the US.

In [ ]:
# Count the occurrences of each industry
industry_counts = df_filtered['Industry'].value_counts()

# Filter for the top 10 industries
top_10_industries = industry_counts.head(10).index

# Filter the dataframe to include only rows from the top 10 industries
df_top_10_industries = df_filtered[df_filtered['Industry'].isin(top_10_industries)]

# Display the filtered dataframe with the top 10 industries
df_top_10_industries[['Industry']].head()

# Box plot of ratings by top industries
plt.figure(figsize=(10, 6))
sns.boxplot(x='Industry', y='Rating', data=df_top_10_industries)
plt.title('Company Ratings by Industry')
plt.xlabel('Industry')
plt.ylabel('Company Rating')
plt.xticks(rotation=90)
plt.show()

The box plot shows the distribution of company ratings across the top 10 industries. Most industries have median ratings around 3.8 to 4.0, with some variability in the distribution. This indicates that while most industries maintain good company ratings, there are some differences in the spread of these ratings.

<b>Most In-Demand Skills:</b><br>
<b>Top Skills:</b> Analytics, Machine learning, Analysis, AI, Modeling, Python, Statistics, Statistical analysis, AWS, SQL, Data visualization, SAS, Data analysis, Tableau, Data mining.<br>
These skills are critical for candidates looking to secure a Data Scientist or Data Analyst position.<br><br>
<b>Top Locations for Jobs:</b><br>
<b>Top Locations:</b> Major states with tech hubs have the most job opportunities.<br>
Visualization shows the distribution of these jobs across each state.<br><br>
<b>Top Industries Hiring:</b><br>
<b>Key Industries:</b> Information Technology, Finance, Healthcare, Enterprise Software & Network Solutions, and Aerospace & Defense.
Visualization shows the distribution of these jobs across various industries.<br><br>
<b>Data Relationships:</b><br>
<b>Scatter Plot:</b> Weak positive correlation between average salary estimates and industry.<br>
<b>Box Plots:</b> Similar distributions of company ratings across job titles, locations, and industries.<br><br>
<b>Feature Engineering:</b><br>
Categorical features have been one-hot encoded.<br>
Numerical features have been standardized.<br><br>
<b>Recommendations</b><br>
<b>1. Skill Development:</b> Prioritize learning Machine learning, Analytics, Python, Statistics, SQL, and Data visualization. These are the some of the most frequently mentioned skills in job descriptions.<br>
<b>2. Targeted Job Search:</b> Focus on the top locations and industries identified. This will increase the efficiency of your job search<br>
<b>3. Continuous Analysis:</b> Regularly update the analysis to capture trends and changes in the job market.

<b>Build 3 different models to determine which is best.</b> 1. Logistic Regression with One-Hot Encoding, 2. Random Forest Classifier with Label Encoding, 3. Gradient Boosting (LightGBM, CatBoost)

In [ ]:
# Identify all unique skills from the 'Extracted Skills' column
unique_skills = set(skill for skills_list in df_filtered['Extracted Skills'].dropna().str.split(', ') for skill in skills_list)

# Create binary columns for each skill (multi-label target)
for skill in unique_skills:
    df_filtered[f'Target_{skill}'] = df_filtered['Extracted Skills'].apply(lambda x: 1 if skill in x else 0)

# Define the target variables (multi-label format)
y = df_filtered[[f'Target_{skill}' for skill in unique_skills]]

# Check and remove skills that only have one class (all 0s or all 1s)
valid_skills = [col for col in y.columns if y[col].nunique() > 1]

# Filter the target dataframe to only include valid skills (with both 0s and 1s)
y_valid = y[valid_skills]

# Define features (e.g., 'Rating', 'State', 'Industry')
X = df_filtered[['Rating', 'State', 'Industry']]

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=['State', 'Industry'], drop_first=True)

# Handle missing values in 'Rating' by filling with the mean
X_encoded['Rating'].fillna(X_encoded['Rating'].mean(), inplace=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_valid, test_size=0.2, random_state=42)

# Train a multi-output logistic regression model (one model per skill)
log_reg = LogisticRegression(max_iter=1000)
multi_target_model = MultiOutputClassifier(log_reg, n_jobs=-1)
multi_target_model.fit(X_train, y_train)

# Make predictions for each skill and evaluate the models
y_pred = multi_target_model.predict(X_test)

# Calculate accuracy for each skill
accuracies = (y_pred == y_test).mean(axis=0)

# Display the accuracy for each skill
skill_accuracy = pd.DataFrame({'Skill': valid_skills, 'Accuracy': accuracies})
print(skill_accuracy.sort_values(by='Accuracy', ascending=False))

In [ ]:
# Evaluate the model
# Initialize dictionaries to store the evaluation metrics for each skill
evaluation_metrics = {
    'Skill': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'Confusion Matrix': []
}

# Loop through each skill (target column) to compute the metrics
for skill in valid_skills:
    # Predictions and true values for the skill
    y_pred_skill = y_pred[:, y_valid.columns.get_loc(skill)]
    y_true_skill = y_test[skill].values
    
    # Accuracy
    acc = accuracy_score(y_true_skill, y_pred_skill)
    
    # Precision, Recall, F1 Score - set to zero if no positive predictions
    precision = precision_score(y_true_skill, y_pred_skill, zero_division=0)
    recall = recall_score(y_true_skill, y_pred_skill, zero_division=0)
    f1 = f1_score(y_true_skill, y_pred_skill, zero_division=0)
    
    # Confusion Matrix
    cm = confusion_matrix(y_true_skill, y_pred_skill)
    
    # Store the metrics
    evaluation_metrics['Skill'].append(skill)
    evaluation_metrics['Accuracy'].append(acc)
    evaluation_metrics['Precision'].append(precision)
    evaluation_metrics['Recall'].append(recall)
    evaluation_metrics['F1 Score'].append(f1)
    evaluation_metrics['Confusion Matrix'].append(cm)

# Convert the metrics dictionary to a DataFrame for easy viewing
evaluation_df = pd.DataFrame(evaluation_metrics)

# Display the evaluation metrics
print(evaluation_df[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])

# Print the confusion matrix for each skill
for skill, cm in zip(evaluation_metrics['Skill'], evaluation_metrics['Confusion Matrix']):
    print(f"\nConfusion Matrix for {skill}:\n{cm}")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Tune model for better performance
# Define a grid of regularization strength values
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize a list to store the best models for each skill
best_logistic_models = {}

# Loop through each skill (multi-label target)
for skill in valid_skills:
    print(f"Running GridSearch for {skill}...")
    
    # Create a Logistic Regression instance
    log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')
    
    # Set up GridSearchCV for the current skill (1D target)
    grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='f1_macro')
    
    # Fit GridSearchCV for the current skill
    grid_search.fit(X_train, y_train[skill])
    
    # Store the best model for the current skill
    best_logistic_models[skill] = grid_search.best_estimator_
    
    # Print the best hyperparameters for the current skill
    print(f"Best parameters for {skill}: {grid_search.best_params_}")

# After the loop, we have the best logistic regression models for each skill

In [ ]:
# Use the best C values from the grid search
best_params = {
    'Target_Tableau': 0.1,
    'Target_Hadoop': 100,
    'Target_Deep Learning': 1,
    'Target_Spark': 10,
    'Target_TensorFlow': 100,
    'Target_Machine Learning': 100,
    'Target_scikit-learn': 100,
    'Target_Data Analysis': 100,
    'Target_Keras': 100,
    'Target_Excel': 100,
    'Target_AWS': 1,
    'Target_SQL': 0.01,
    'Target_Cloud': 10,
    'Target_NLP': 1,
    'Target_Data Mining': 0.1,
    'Target_Power BI': 100,
    'Target_R': 0.01,
    'Target_Python': 100,
    'Target_Azure': 1,
    'Target_Statistics': 10
}

# Initialize the logistic regression models using the best parameters
logistic_models = {}
for skill, C_value in best_params.items():
    logistic_models[skill] = LogisticRegression(C=C_value, max_iter=1000, class_weight='balanced')

# Train each model on the respective target skill
for skill, model in logistic_models.items():
    print(f"Training model for {skill}...")
    model.fit(X_train, y_train[skill])

# Make predictions and evaluate the models for each skill
evaluation_metrics = {'Skill': [], 'Accuracy': [], 'Precision': [], 'Recall': [], 'F1 Score': [], 'Confusion Matrix': []}

for skill, model in logistic_models.items():
    print(f"Evaluating model for {skill}...")
    y_pred_skill = model.predict(X_test)
    y_true_skill = y_test[skill].values

    # Calculate evaluation metrics
    acc = accuracy_score(y_true_skill, y_pred_skill)
    precision = precision_score(y_true_skill, y_pred_skill, zero_division=0)
    recall = recall_score(y_true_skill, y_pred_skill, zero_division=0)
    f1 = f1_score(y_true_skill, y_pred_skill, zero_division=0)
    cm = confusion_matrix(y_true_skill, y_pred_skill)

    # Store the metrics
    evaluation_metrics['Skill'].append(skill)
    evaluation_metrics['Accuracy'].append(acc)
    evaluation_metrics['Precision'].append(precision)
    evaluation_metrics['Recall'].append(recall)
    evaluation_metrics['F1 Score'].append(f1)
    evaluation_metrics['Confusion Matrix'].append(cm)

# Convert the metrics dictionary to a DataFrame for easy viewing
evaluation_df = pd.DataFrame(evaluation_metrics)

# Display the evaluation metrics
print(evaluation_df[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])

<h3>Summary:</h3>
<b>Improvements:</b> Tuning the logistic regression model resulted in improved F1 scores for most skills, particularly those that had poor scores in the original model. Skills such as Target_Machine Learning, Target_Tableau, and Target_Statistics benefited the most from the tuning.

<b>Challenges with Rare Skills:</b> Despite tuning, skills with fewer positive examples (e.g., Target_Keras, Target_Power BI) still exhibit poor performance due to class imbalance.

<b>Trade-offs:</b> While tuning improved performance for many skills, some skills like Target_Data Analysis and Target_R saw slight decreases in performance, indicating a potential trade-off between precision and recall or overfitting in certain areas.

To further improve, additional techniques like SMOTE (Synthetic Minority Over-sampling Technique) or undersampling for imbalanced classes could be applied, especially for skills with few positive examples.

In [ ]:
# Random Forest Classifier with label encoding
# Instantiate the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Wrap it in MultiOutputClassifier to handle multi-label classification
multi_rf_model = MultiOutputClassifier(rf_model, n_jobs=-1)

# Train the Random Forest model on the training data
multi_rf_model.fit(X_train, y_train)

# Make predictions for each skill on the test set
y_pred_rf = multi_rf_model.predict(X_test)

# Initialize dictionaries to store the evaluation metrics for each skill
rf_evaluation_metrics = {
    'Skill': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'Confusion Matrix': []
}

# Loop through each skill (target column) to compute the metrics
for skill in valid_skills:
    y_pred_skill_rf = y_pred_rf[:, y_valid.columns.get_loc(skill)]
    y_true_skill_rf = y_test[skill].values
    
    acc_rf = accuracy_score(y_true_skill_rf, y_pred_skill_rf)
    precision_rf = precision_score(y_true_skill_rf, y_pred_skill_rf, zero_division=0)
    recall_rf = recall_score(y_true_skill_rf, y_pred_skill_rf, zero_division=0)
    f1_rf = f1_score(y_true_skill_rf, y_pred_skill_rf, zero_division=0)
    cm_rf = confusion_matrix(y_true_skill_rf, y_pred_skill_rf)
    
    # Store the metrics
    rf_evaluation_metrics['Skill'].append(skill)
    rf_evaluation_metrics['Accuracy'].append(acc_rf)
    rf_evaluation_metrics['Precision'].append(precision_rf)
    rf_evaluation_metrics['Recall'].append(recall_rf)
    rf_evaluation_metrics['F1 Score'].append(f1_rf)
    rf_evaluation_metrics['Confusion Matrix'].append(cm_rf)

# Convert the metrics dictionary to a DataFrame for easy viewing
rf_evaluation_df = pd.DataFrame(rf_evaluation_metrics)

# Display the evaluation metrics
print(rf_evaluation_df[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])

In [ ]:
# Use class_weight='balanced' to help with the imbalanced classes.
# Instantiate the Random Forest model with class_weight='balanced'
rf_model_balanced = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Wrap it in MultiOutputClassifier to handle multi-label classification
multi_rf_model_balanced = MultiOutputClassifier(rf_model_balanced, n_jobs=-1)

# Train the Random Forest model on the training data
multi_rf_model_balanced.fit(X_train, y_train)

# Make predictions for each skill on the test set
y_pred_rf_balanced = multi_rf_model_balanced.predict(X_test)

# Initialize a dictionary to store the evaluation metrics
rf_evaluation_metrics = {
    'Skill': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'Confusion Matrix': []
}

# Loop through each skill (target column) to compute the metrics
valid_skills = y_train.columns  # Assuming your y_train DataFrame contains skill names as columns
for skill in valid_skills:
    y_pred_skill_rf = y_pred_rf_balanced[:, y_test.columns.get_loc(skill)]
    y_true_skill_rf = y_test[skill].values
    
    # Calculate metrics for the current skill
    acc_rf = accuracy_score(y_true_skill_rf, y_pred_skill_rf)
    precision_rf = precision_score(y_true_skill_rf, y_pred_skill_rf, zero_division=0)
    recall_rf = recall_score(y_true_skill_rf, y_pred_skill_rf, zero_division=0)
    f1_rf = f1_score(y_true_skill_rf, y_pred_skill_rf, zero_division=0)
    cm_rf = confusion_matrix(y_true_skill_rf, y_pred_skill_rf)
    
    # Store the metrics in the dictionary
    rf_evaluation_metrics['Skill'].append(skill)
    rf_evaluation_metrics['Accuracy'].append(acc_rf)
    rf_evaluation_metrics['Precision'].append(precision_rf)
    rf_evaluation_metrics['Recall'].append(recall_rf)
    rf_evaluation_metrics['F1 Score'].append(f1_rf)
    rf_evaluation_metrics['Confusion Matrix'].append(cm_rf)

# Convert the metrics dictionary to a DataFrame for easy viewing
rf_evaluation_df = pd.DataFrame(rf_evaluation_metrics)

# Display the evaluation metrics
print(rf_evaluation_df[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])

<h3>Conclusion:</h3>
The tuned Random Forest model with class balancing generally shows improvements in recall and F1 scores, particularly for difficult-to-predict classes. The accuracy remains stable overall. However, precision dropped for a few cases, possibly due to the class balancing leading to more false positives. The model performed better for skills like Target_Spark, Target_NLP, and Target_Data Mining, but it still struggles with certain skills like Target_Azure, Target_Keras, and Target_Power BI.

In [ ]:
# Define a function to clean feature names by replacing special characters with underscores
def clean_feature_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)  # Replace special characters with '_'
    return df

# Clean feature names in both training and test sets
X_train_cleaned = clean_feature_names(X_train)
X_test_cleaned = clean_feature_names(X_test)

# Define the LightGBM model
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', random_state=42)

# Wrap it in MultiOutputClassifier to handle multi-label classification
multi_lgb_model = MultiOutputClassifier(lgb_model, n_jobs=-1)

# Train the LightGBM model on the cleaned training data
multi_lgb_model.fit(X_train_cleaned, y_train)

# Make predictions for each skill on the cleaned test set
y_pred_lgb = multi_lgb_model.predict(X_test_cleaned)

# Step 7: Initialize dictionaries to store the evaluation metrics for each skill
lgb_evaluation_metrics = {
    'Skill': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'Confusion Matrix': []
}

# Loop through each skill (target column) to compute the metrics
for skill in valid_skills:
    y_pred_skill_lgb = y_pred_lgb[:, y_valid.columns.get_loc(skill)]
    y_true_skill_lgb = y_test[skill].values
    
    acc_lgb = accuracy_score(y_true_skill_lgb, y_pred_skill_lgb)
    precision_lgb = precision_score(y_true_skill_lgb, y_pred_skill_lgb, zero_division=0)
    recall_lgb = recall_score(y_true_skill_lgb, y_pred_skill_lgb, zero_division=0)
    f1_lgb = f1_score(y_true_skill_lgb, y_pred_skill_lgb, zero_division=0)
    cm_lgb = confusion_matrix(y_true_skill_lgb, y_pred_skill_lgb)
    
    # Store the metrics
    lgb_evaluation_metrics['Skill'].append(skill)
    lgb_evaluation_metrics['Accuracy'].append(acc_lgb)
    lgb_evaluation_metrics['Precision'].append(precision_lgb)
    lgb_evaluation_metrics['Recall'].append(recall_lgb)
    lgb_evaluation_metrics['F1 Score'].append(f1_lgb)
    lgb_evaluation_metrics['Confusion Matrix'].append(cm_lgb)

# Convert the metrics dictionary to a DataFrame for easy viewing
lgb_evaluation_df = pd.DataFrame(lgb_evaluation_metrics)

# Display the evaluation metrics
print(lgb_evaluation_df[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])

In [ ]:
!pip install xgboost


In [ ]:
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

# Define the individual base models (with class_weight for imbalance handling)
lgb_model_2 = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', class_weight='balanced', random_state=42)
xgb_model = XGBClassifier(objective='binary:logistic', random_state=42, use_label_encoder=False, eval_metric='logloss')

# Define the meta-model (Logistic Regression in this case) for stacking
meta_model = LogisticRegression()

# Define the stacking classifier
stacking_model = StackingClassifier(
    estimators=[
        ('lgb', lgb_model_2),
        ('xgb', xgb_model)
    ],
    final_estimator=meta_model,
    n_jobs=-1
)

# Wrap it in MultiOutputClassifier
multi_output_stacking_model = MultiOutputClassifier(stacking_model, n_jobs=-1)

# Hyperparameter grid for LightGBM
lgb_param_grid = {
    'estimator__lgb__n_estimators': [50, 100, 200],
    'estimator__lgb__learning_rate': [0.01, 0.1, 0.3],
    'estimator__lgb__num_leaves': [20, 31, 40],
    'estimator__lgb__max_depth': [-1, 5, 10],
    'estimator__lgb__min_child_samples': [10, 20, 30],
    'estimator__lgb__subsample': [0.7, 0.8, 1.0]
}

# Hyperparameter grid for XGBoost
xgb_param_grid = {
    'estimator__xgb__n_estimators': [50, 100, 200],
    'estimator__xgb__learning_rate': [0.01, 0.1, 0.3],
    'estimator__xgb__max_depth': [3, 5, 7],
    'estimator__xgb__min_child_weight': [1, 5, 10],
    'estimator__xgb__subsample': [0.7, 0.8, 1.0],
    'estimator__xgb__colsample_bytree': [0.7, 0.8, 1.0]
}

# Hyperparameter grid for Logistic Regression (meta-model)
lr_param_grid = {
    'estimator__final_estimator__C': np.logspace(-4, 4, 10),
    'estimator__final_estimator__penalty': ['l2'],
    'estimator__final_estimator__solver': ['liblinear', 'lbfgs']
}

# Combine the parameter grids into a search space
param_grid = {**lgb_param_grid, **xgb_param_grid, **lr_param_grid}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(multi_output_stacking_model, param_grid, n_iter=50, scoring='f1_weighted', cv=3, verbose=1, random_state=42, n_jobs=-1)

# Fit the model with hyperparameter tuning
random_search.fit(X_train_cleaned, y_train)

# Print the best parameters found by RandomizedSearchCV
print("Best Hyperparameters:", random_search.best_params_)

# Make predictions with the tuned model
y_pred_stacked_tuned = random_search.predict(X_test_cleaned)

# Initialize dictionaries to store the evaluation metrics for each skill
stacked_evaluation_metrics = {
    'Skill': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'Confusion Matrix': []
}

# Assuming `valid_skills` is the list of column names representing each skill
for i, skill in enumerate(valid_skills):  # Iterate over the skills
    # Get predictions and true values for the current skill
    y_pred_skill_stacked = y_pred_stacked[:, i]  # Access the i-th skill's predictions
    y_true_skill_stacked = y_test[:, i]  # Access the i-th skill's true values

    # Calculate metrics for the current skill
    acc_stacked = accuracy_score(y_true_skill_stacked, y_pred_skill_stacked)
    precision_stacked = precision_score(y_true_skill_stacked, y_pred_skill_stacked, zero_division=0)
    recall_stacked = recall_score(y_true_skill_stacked, y_pred_skill_stacked, zero_division=0)
    f1_stacked = f1_score(y_true_skill_stacked, y_pred_skill_stacked, zero_division=0)
    cm_stacked = confusion_matrix(y_true_skill_stacked, y_pred_skill_stacked)

    # Store the metrics
    stacked_evaluation_metrics['Skill'].append(skill)
    stacked_evaluation_metrics['Accuracy'].append(acc_stacked)
    stacked_evaluation_metrics['Precision'].append(precision_stacked)
    stacked_evaluation_metrics['Recall'].append(recall_stacked)
    stacked_evaluation_metrics['F1 Score'].append(f1_stacked)
    stacked_evaluation_metrics['Confusion Matrix'].append(cm_stacked)

# Convert the metrics dictionary to a DataFrame for easy viewing
stacked_evaluation_df = pd.DataFrame(stacked_evaluation_metrics)

# Display the evaluation metrics
print(stacked_evaluation_df[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the stacking classifier
stacking_model = StackingClassifier(
    estimators=[
        ('lgb', lgb_model_2),
        ('xgb', xgb_model)
    ],
    final_estimator=meta_model,
    n_jobs=-1
)

# Wrap the stacking model in MultiOutputClassifier
multi_output_stacking_model = MultiOutputClassifier(stacking_model, n_jobs=-1)

# Hyperparameter grid for LightGBM
lgb_param_grid = {
    'estimator__lgb__n_estimators': [50, 100, 200],
    'estimator__lgb__learning_rate': [0.01, 0.1, 0.3],
    'estimator__lgb__num_leaves': [20, 31, 40],
    'estimator__lgb__max_depth': [-1, 5, 10],
    'estimator__lgb__min_child_samples': [10, 20, 30],
    'estimator__lgb__subsample': [0.7, 0.8, 1.0]
}

# Hyperparameter grid for XGBoost
xgb_param_grid = {
    'estimator__xgb__n_estimators': [50, 100, 200],
    'estimator__xgb__learning_rate': [0.01, 0.1, 0.3],
    'estimator__xgb__max_depth': [3, 5, 7],
    'estimator__xgb__min_child_weight': [1, 5, 10],
    'estimator__xgb__subsample': [0.7, 0.8, 1.0],
    'estimator__xgb__colsample_bytree': [0.7, 0.8, 1.0]
}

# Hyperparameter grid for Logistic Regression (meta-model)
lr_param_grid = {
    'estimator__final_estimator__C': np.logspace(-4, 4, 10),
    'estimator__final_estimator__penalty': ['l2'],
    'estimator__final_estimator__solver': ['liblinear', 'lbfgs']
}

# Combine the parameter grids into a single search space
param_grid = {**lgb_param_grid, **xgb_param_grid, **lr_param_grid}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(multi_output_stacking_model, param_grid, n_iter=50, scoring='f1_weighted', cv=3, verbose=1, random_state=42, n_jobs=-1)

# Fit the model with hyperparameter tuning
random_search.fit(X_train_cleaned, y_train)

# Print the best parameters found by RandomizedSearchCV
print("Best Hyperparameters:", random_search.best_params_)

# Make predictions with the tuned model
y_pred_stacked_tuned = random_search.predict(X_test_cleaned)

# Initialize dictionaries to store the evaluation metrics for each skill
stacked_evaluation_metrics = {
    'Skill': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'Confusion Matrix': []
}

# Assuming `valid_skills` is the list of column names representing each skill
for i, skill in enumerate(valid_skills):  # Iterate over the skills
    # Get predictions and true values for the current skill
    y_pred_skill_stacked = y_pred_stacked_tuned[:, i]  # Access the i-th skill's predictions
    y_true_skill_stacked = y_test[:, i]  # Access the i-th skill's true values

    # Calculate metrics for the current skill
    acc_stacked = accuracy_score(y_true_skill_stacked, y_pred_skill_stacked)
    precision_stacked = precision_score(y_true_skill_stacked, y_pred_skill_stacked, zero_division=0)
    recall_stacked = recall_score(y_true_skill_stacked, y_pred_skill_stacked, zero_division=0)
    f1_stacked = f1_score(y_true_skill_stacked, y_pred_skill_stacked, zero_division=0)
    cm_stacked = confusion_matrix(y_true_skill_stacked, y_pred_skill_stacked)

    # Store the metrics
    stacked_evaluation_metrics['Skill'].append(skill)
    stacked_evaluation_metrics['Accuracy'].append(acc_stacked)
    stacked_evaluation_metrics['Precision'].append(precision_stacked)
    stacked_evaluation_metrics['Recall'].append(recall_stacked)
    stacked_evaluation_metrics['F1 Score'].append(f1_stacked)
    stacked_evaluation_metrics['Confusion Matrix'].append(cm_stacked)

# Convert the metrics dictionary to a DataFrame for easy viewing
stacked_evaluation_df_2 = pd.DataFrame(stacked_evaluation_metrics)

# Display the evaluation metrics
print(stacked_evaluation_df_2[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])


In [ ]:
print(stacked_evaluation_df_2[['Skill', 'Accuracy', 'Precision', 'Recall', 'F1 Score']])

<h3>Conclusion:</h3>

The tuned model shows some overall improvements, particularly in skills like Target_Deep Learning, but also experiences minor trade-offs in precision and recall for certain skills. The model has become more balanced, with some skills seeing better F1 scores at the expense of slight declines in precision or recall. Further fine-tuning could address these marginal trade-offs.

<h3>Summary of all 3 models:</h3>

<b>Logistic Regression:</b> Best for environments where simplicity, speed, and interpretability are critical, but it does not perform well for complex, non-linear problems.

<b>Random Forest:</b> A strong middle ground, offering good performance for a wide variety of tasks while being more scalable and efficient than the stacked model. It balances complexity and predictive power, making it a good general-purpose model, especially when computational resources are available.

<b>Stacked Model (LightGBM + XGBoost):</b> Best for extracting maximum performance in complex problems, particularly for tasks like Deep Learning and Machine Learning where it showed improvement. However, the high computational cost and maintenance requirements make it suitable for environments with abundant computational resources and the ability to handle model complexity.

<h3>Recommendation:</h3>

For most use cases, especially where performance and computational efficiency are both important, **Random Forest** is the best overall option. It provides a good balance of performance, scalability, and maintenance cost.
**Stacked Model** is preferable when the highest predictive accuracy is required, particularly for complex non-linear problems, and when computational resources and time for tuning are not limiting factors.